# Session 3 — Understanding Agent Architectures
Focus: Compare the core reasoning engines and architectures for building agents

In [ ]:
# Install required packages for agent implementations
!pip install langchain-experimental wikipedia

### 1. Agent Types: ReAct, Plan-and-Execute, Self-Ask

**ReAct (Reason + Act) Agents:**
Think of ReAct agents like a detective solving a case. They:
1. **Reason**: Think about what they know and what they need to find out
2. **Act**: Take action to gather more information (use tools, search, etc.)
3. **Repeat**: Continue this cycle until they solve the problem

**Key Characteristics:**
- Iterative approach to problem-solving
- Dynamic adaptation to new information
- Good for open-ended problems where the path isn't clear
- Can handle unexpected obstacles during execution

**Plan-and-Execute Agents:**
These work like project managers:
1. **Plan**: Create a detailed step-by-step plan first
2. **Execute**: Carry out each step of the plan systematically
3. **Adjust**: Modify the plan if obstacles are encountered

**Key Characteristics:**
- Structured, methodical approach
- Better for complex, multi-step problems
- Clear separation between planning and execution
- Easier to debug and understand the thought process

**Self-Ask Agents:**
These operate like curious students:
1. **Question**: Identify what they need to know
2. **Research**: Seek out answers to their questions
3. **Synthesize**: Combine the information to form a complete answer

**Key Characteristics:**
- Focused on information gathering and synthesis
- Excellent for research-oriented tasks
- Good at breaking down complex questions into sub-questions
- Natural approach for fact-finding missions

**Architecture Comparison Table:**

| Aspect | ReAct | Plan-and-Execute | Self-Ask |
|--------|-------|------------------|----------|
| Approach | Iterative reasoning | Pre-planned execution | Question-based |
| Best For | Dynamic problems | Complex workflows | Research tasks |
| Complexity | Medium | High | Medium |
| Reliability | Good | Excellent | Good |
| Speed | Fast for simple tasks | Slower but thorough | Medium |
| Error Handling | Adaptive | Robust | Question-based |

### 3. Trade-offs in Complexity and Reliability

**The Complexity-Reliability Spectrum:**
Agent design involves balancing complexity against reliability. More complex agents can handle more sophisticated tasks but introduce more potential failure points.

**Key Trade-offs:**

1. **Flexibility vs. Predictability**
   - Complex agents: More flexible but less predictable
   - Simple agents: More predictable but less flexible

2. **Capability vs. Reliability**
   - More tools = more capabilities but more failure points
   - Simpler architectures = fewer features but higher reliability

3. **Development Time vs. Functionality**
   - Complex agents require more development and testing time
   - Simple agents can be deployed quickly but with limited functionality

4. **Cost vs. Performance**
   - Complex agents typically use more API calls and computing resources
   - Simple agents are more cost-effective for specific tasks

**Reliability Engineering Patterns:**

1. **Circuit Breaker Pattern**:
   - Temporarily disable failing tools to prevent cascading failures
   - Implement retry mechanisms with exponential backoff

2. **Validation Layers**:
   - Validate tool inputs before execution
   - Verify tool outputs before using them in subsequent steps

3. **Fallback Strategies**:
   - Alternative tools for critical operations
   - Graceful degradation when tools are unavailable

4. **Monitoring and Alerting**:
   - Track tool success/failure rates
   - Set up alerts for abnormal patterns

**Performance Considerations:**

1. **Latency Management**:
   - Cache frequent tool responses
   - Implement parallel tool execution where possible
   - Set reasonable timeouts for each tool

2. **Cost Optimization**:
   - Monitor API usage and costs
   - Implement usage quotas and rate limiting
   - Use cheaper tools for simple operations

3. **Scalability Planning**:
   - Design for horizontal scaling
   - Consider stateless architectures where possible
   - Implement efficient resource utilization

## Hands-On: Implement a simple task using both ReAct and Plan-and-Execute agents

In [6]:
# First, let's set up our LLM and tools
from langchain.agents import AgentType, initialize_agent, Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.utilities import WikipediaAPIWrapper
from langchain.chains import LLMMathChain
from langchain import SerpAPIWrapper
import os

# Initialize our LLM (using OpenRouter as before)
from langchain_openai import ChatOpenAI
api_key ='sk-or-v1-6173516c9ecc07d1e10b606a570d1e73190a53974895d204a9fc40bb92a54c0fa'  #remove the 'a' at the end to use the api key
llm = ChatOpenAI(
    model="meta-llama/llama-3.1-8b-instruct",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key,
    temperature=0,
    max_tokens=500,
)

# Create tools for our agents
# Note: For simplicity, we'll use simulated tools since some APIs require keys
# In a real scenario, you'd use actual tool implementations

# Simulated search tool
def simulated_search(query):
    """Simulate a search tool that returns predefined results"""
    knowledge_base = {
        "current temperature in London": "15°C, cloudy with a chance of rain",
        "population of France": "67.5 million people",
        "CEO of Microsoft": "Satya Nadella",
        "capital of Japan": "Tokyo",
        "height of Eiffel Tower": "330 meters including antenna",
        "foundation date of Google": "September 4, 1998",
        "current president of United States": "Joe Biden",
        "distance from Earth to Moon": "384,400 kilometers on average",
    }
    return knowledge_base.get(query.lower(), f"No results found for: {query}")

# Simulated calculator tool
def simulated_calculator(expression):
    """Simulate a calculator tool"""
    try:
        # Simple safe evaluation
        if "+" in expression:
            parts = expression.split("+")
            return str(float(parts[0]) + float(parts[1]))
        elif "-" in expression:
            parts = expression.split("-")
            return str(float(parts[0]) - float(parts[1]))
        elif "*" in expression:
            parts = expression.split("*")
            return str(float(parts[0]) * float(parts[1]))
        elif "/" in expression:
            parts = expression.split("/")
            return str(float(parts[0]) / float(parts[1]))
        elif "sqrt" in expression:
            num = float(expression.replace("sqrt", ""))
            return str(num ** 0.5)
        else:
            return "Cannot evaluate this expression"
    except:
        return "Error evaluating expression"

# Simulated unit conversion tool
def simulated_converter(query):
    """Simulate a unit conversion tool"""
    conversions = {
        "celsius to fahrenheit": lambda x: (float(x) * 9/5) + 32,
        "meters to feet": lambda x: float(x) * 3.28084,
        "kilometers to miles": lambda x: float(x) * 0.621371,
        "kilograms to pounds": lambda x: float(x) * 2.20462,
    }

    for conversion, func in conversions.items():
        if conversion in query.lower():
            try:
                # Extract the number from the query
                import re
                numbers = re.findall(r"[-+]?\d*\.\d+|\d+", query)
                if numbers:
                    result = func(numbers[0])
                    return f"{numbers[0]} in {conversion.split()[-1]} is {result:.2f}"
            except:
                pass

    return "Conversion not supported or invalid input"

# Create tool objects
tools = [
    Tool(
        name="Search",
        func=simulated_search,
        description="Useful for answering questions about current events, facts, and general knowledge. Input should be a clear search query."
    ),
    Tool(
        name="Calculator",
        func=simulated_calculator,
        description="Useful for performing mathematical calculations. Input should be a mathematical expression."
    ),
    Tool(
        name="UnitConverter",
        func=simulated_converter,
        description="Useful for converting between units. Input should specify the conversion type and value."
    )
]

# Create a ReAct agent
react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5  # Prevent infinite loops
)

# Create a Plan-and-Execute agent
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
plan_and_execute_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# Test both agents with the same task
task = "What is the current temperature in London, and what would that be in Fahrenheit? First convert Celsius to Fahrenheit."

print("=" * 80)
print("REACT AGENT EXECUTION:")
print("=" * 80)
react_result = react_agent.run(task)

print("\n" + "=" * 80)
print("PLAN-AND-EXECUTE AGENT EXECUTION:")
print("=" * 80)
plan_and_execute_result = plan_and_execute_agent.run(task)

print("\n" + "=" * 80)
print("COMPARISON RESULTS:")
print("=" * 80)
print("ReAct Agent Result:", react_result)
print("Plan-and-Execute Agent Result:", plan_and_execute_result)

REACT AGENT EXECUTION:


> Entering new AgentExecutor chain...
Thought: I need to find the current temperature in London and convert it from Celsius to Fahrenheit.
Action: Search
Action Input: current temperature in London
Observation: No results found for: current temperature in London
Thought:It seems that the search query didn't yield any results. Let me try a different approach.

Thought: I need to find a reliable source for the current temperature in London.
Action: Search
Action Input: current weather in London
Observation: No results found for: current weather in London
Thought:Thought: It seems that the search query didn't yield any results. Let me try a different approach.

Thought: I need to find a reliable source for the current temperature in London.
Action: Search
Action Input: current temperature in London
Observation: No results found for: current temperature in London
Thought:Thought: I need to find a reliable source for the current temperature in London.
Action: Search

## Comparing Agent Architectures

In [8]:
# Let's create a more detailed comparison function
def compare_agents(task, agents, agent_names):
    """Compare multiple agents on the same task"""
    results = {}

    for name, agent in zip(agent_names, agents):
        print(f"\n{'-'*40}")
        print(f"{name} AGENT:")
        print(f"{'-'*40}")
        try:
            result = agent.run(task)
            results[name] = {
                "result": result,
                "error": None
            }
        except Exception as e:
            results[name] = {
                "result": None,
                "error": str(e)
            }
            print(f"Error: {e}")

    return results

# Create a specialized search tool for Self-Ask agent
self_ask_search_tool = Tool(
    name="Intermediate Answer",
    func=simulated_search,
    description="Useful for when you need to ask with search."
)

# Create a Self-Ask agent with only the search tool
self_ask_agent = initialize_agent(
    [self_ask_search_tool],
    llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    handle_parsing_errors=True
)

# Compare all three agent types
agents = [react_agent, plan_and_execute_agent, self_ask_agent]
agent_names = ["ReAct", "Plan-and-Execute", "Self-Ask"]

comparison_task = "Who is the CEO of Microsoft and what is the population of France?"
results = compare_agents(comparison_task, agents, agent_names)

# Display comparison results
print("\n" + "=" * 80)
print("AGENT COMPARISON SUMMARY:")
print("=" * 80)
for name in agent_names:
    print(f"\n{name}:")
    if results[name]["error"]:
        print(f"  Error: {results[name]['error']}")
    else:
        print(f"  Result: {results[name]['result']}")


----------------------------------------
ReAct AGENT:
----------------------------------------


> Entering new AgentExecutor chain...
Thought: I need to find the CEO of Microsoft and the population of France.
Action: Search
Action Input: CEO of Microsoft and population of France
Observation: No results found for: CEO of Microsoft and population of France
Thought:Thought: It seems like the search query was not specific enough. I need to search for the CEO of Microsoft separately and the population of France separately.
Action: Search
Action Input: CEO of Microsoft
Observation: No results found for: CEO of Microsoft
Thought:It seems like I made a mistake in my previous search query. Let me try again.

Action: Search
Action Input: current CEO of Microsoft
Observation: No results found for: current CEO of Microsoft
Thought:Thought: I need to find the current CEO of Microsoft.
Action: Search
Action Input: current CEO of Microsoft
Observation: No results found for: current CEO of Microsoft

### Case Study: Selecting the right agent architecture for different business needs

**Introduction:**
In this case study, we'll explore how different agent architectures suit various business requirements. The choice of agent architecture can significantly impact the effectiveness, reliability, and cost of AI-powered solutions.

**Scenario 1: Simple Data Lookup Agent**
*Business Need:* A customer service chatbot that answers FAQs about product specifications and pricing.

*Requirements:*
- Fast response times
- High reliability
- Simple, predictable queries
- Low maintenance

*Recommended Architecture:* **Single-tool ReAct Agent**
- Why: Simple queries don't require complex planning
- Benefits: Fast responses, easy to implement and maintain
- Tools: Product database lookup, pricing calculator

*Implementation Example:*
```python
# Simple product lookup agent
product_agent = initialize_agent(
    [product_lookup_tool, pricing_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

## Scenario 2: Complex Research Assistant
**Business Need**: A research assistant that can gather information from multiple sources, analyze data, and generate comprehensive reports.

**Requirements**:

    Ability to handle multi-step research processes

    Integration with various data sources

    Sophisticated reasoning capabilities

    Robust error handling

**Recommended Architecture:** Multi-tool Plan-and-Execute Agent

    Why: Complex research requires careful planning and execution

    Benefits: Structured approach, better handling of complex workflows

    Tools: Web search, database access, data analysis tools, report generator

Implementation Example:

In [ ]:
# Complex research agent
research_planner = load_chat_planner(llm)
research_executor = load_agent_executor(llm, research_tools, verbose=True)
research_agent = PlanAndExecute(planner=research_planner, executor=research_executor)

## Scenario 3: Fact Verification System
**Business Need**: A system that verifies claims by cross-referencing multiple sources and providing evidence-based assessments.

**Requirements:**

    Ability to break down complex claims into verifiable facts

    Access to multiple information sources

    Transparent reasoning process

    Confidence scoring for verifications

**Recommended Architecture:** Self-Ask Agent with Multiple Verification Tools

    Why: Natural question-answer approach suits fact verification

    Benefits: Clear audit trail of verification steps, good at handling sub-queries

    Tools: Multiple search engines, fact-checking databases, credibility assessment

**Implementation Example:**

In [ ]:
# Fact verification agent
verification_agent = initialize_agent(
    verification_tools,
    llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True
)

## Decision Framework:
**When selecting an agent architecture, consider these factors:**

**Task Complexity:**

        Simple tasks → ReAct agents

        Complex, multi-step tasks → Plan-and-Execute agents

        Research-oriented tasks → Self-Ask agents

**Reliability Requirements:**

        High reliability needs → Simpler architectures with fewer failure points

        Moderate reliability → Balanced architectures with good error handling

**Development Resources:**

        Limited resources → Start with simpler ReAct agents

        Ample resources → Consider more sophisticated architectures

**Performance Needs:**

        Low latency → ReAct agents with optimized tools

        Thoroughness over speed → Plan-and-Execute agents

**Maintenance Considerations:**

        Easy maintenance → Fewer tools, simpler architectures

        Willing to invest in maintenance → More complex, powerful architectures

**Real-World Implementation Tips:**

    Start with the simplest architecture that meets your needs

    Implement comprehensive monitoring from day one

    Design for graceful degradation when tools fail

    Consider hybrid approaches for complex requirements

    Regularly review and optimize tool selection and usage

In [13]:
# Let's create a function to trace and analyze agent execution
def analyze_agent_execution(agent, task, agent_name):
    """Execute an agent and provide detailed analysis of its execution"""
    print(f"\n{'='*60}")
    print(f"{agent_name} AGENT ANALYSIS:")
    print(f"{'='*60}")
    print(f"Task: {task}")

    # For Self-Ask agent, we might need to modify the task
    if agent_name == "Self-Ask":
        # Self-Ask works better with question-style prompts and single-step tasks
        modified_task = f"Answer the following question: {task}"
        print(f"Modified task for Self-Ask: {modified_task}")
        task = modified_task

    # Execute the agent
    try:
        result = agent.run(task)
        print(f"\nResult: {result}")
        print(f"\nExecution successful!")

        # Analyze the execution
        analysis_prompt = f"""
        Analyze the following agent execution:

        Agent Type: {agent_name}
        Task: {task}
        Result: {result}

        Please provide a brief analysis of:
        1. The reasoning process used by the agent
        2. The tools utilized during execution
        3. The effectiveness of the approach for this task
        4. Any potential improvements or alternative approaches

        Analysis:
        """

        analysis = llm.invoke(analysis_prompt)
        print(f"\nAnalysis:\n{analysis.content}")

    except Exception as e:
        print(f"Execution failed: {e}")
        return None

    return result

# For Self-Ask agent, let's use a simpler task that only requires search
self_ask_analysis_task = "What is the height of the Eiffel Tower?"

# Analyze different agents with appropriate tasks
analysis_task = "What is the height of the Eiffel Tower in feet? (1 meter = 3.28 feet)"

analyze_agent_execution(react_agent, analysis_task, "ReAct")
analyze_agent_execution(plan_and_execute_agent, analysis_task, "Plan-and-Execute")
analyze_agent_execution(self_ask_agent, self_ask_analysis_task, "Self-Ask")


ReAct AGENT ANALYSIS:
Task: What is the height of the Eiffel Tower in feet? (1 meter = 3.28 feet)


> Entering new AgentExecutor chain...
Thought: I need to find the height of the Eiffel Tower in meters to be able to convert it to feet.
Action: Search
Action Input: height of the Eiffel Tower in meters
Observation: No results found for: height of the Eiffel Tower in meters
Thought:Thought: I need to try a different search query to find the height of the Eiffel Tower in meters.
Action: Search
Action Input: Eiffel Tower height in meters
Observation: No results found for: Eiffel Tower height in meters
Thought:Thought: I need to try a different search query to find the height of the Eiffel Tower in meters.
Action: Search
Action Input: Eiffel Tower height
Observation: No results found for: Eiffel Tower height
Thought:Thought: I need to try a different search query to find the general information about the Eiffel Tower.
Action: Search
Action Input: Eiffel Tower
Observation: No results found 

'Agent stopped due to iteration limit or time limit.'

## Advanced Agent Tracing and Analysis

In [10]:
# Let's create a function to trace and analyze agent execution
def analyze_agent_execution(agent, task, agent_name):
    """Execute an agent and provide detailed analysis of its execution"""
    print(f"\n{'='*60}")
    print(f"{agent_name} AGENT ANALYSIS:")
    print(f"{'='*60}")
    print(f"Task: {task}")

    # Execute the agent
    try:
        result = agent.run(task)
        print(f"\nResult: {result}")
        print(f"\nExecution successful!")

        # Analyze the execution
        analysis_prompt = f"""
        Analyze the following agent execution:

        Agent Type: {agent_name}
        Task: {task}
        Result: {result}

        Please provide a brief analysis of:
        1. The reasoning process used by the agent
        2. The tools utilized during execution
        3. The effectiveness of the approach for this task
        4. Any potential improvements or alternative approaches

        Analysis:
        """

        analysis = llm.invoke(analysis_prompt)
        print(f"\nAnalysis:\n{analysis.content}")

    except Exception as e:
        print(f"Execution failed: {e}")
        return None

    return result

# Analyze different agents with the same task
analysis_task = "What is the height of the Eiffel Tower in feet? (1 meter = 3.28 feet)"

analyze_agent_execution(react_agent, analysis_task, "ReAct")
analyze_agent_execution(plan_and_execute_agent, analysis_task, "Plan-and-Execute")
analyze_agent_execution(self_ask_agent, analysis_task, "Self-Ask")


ReAct AGENT ANALYSIS:
Task: What is the height of the Eiffel Tower in feet? (1 meter = 3.28 feet)


> Entering new AgentExecutor chain...
Thought: I need to find the height of the Eiffel Tower in meters first, and then convert it to feet.
Action: Search
Action Input: height of the Eiffel Tower in meters
Observation: No results found for: height of the Eiffel Tower in meters
Thought:Thought: I need to rephrase the search query to get accurate results.
Action: Search
Action Input: Eiffel Tower height in meters
Observation: No results found for: Eiffel Tower height in meters
Thought:Thought: I need to try a different search query to get accurate results.
Action: Search
Action Input: Eiffel Tower height
Observation: No results found for: Eiffel Tower height
Thought:Thought: I need to try a more specific search query to get accurate results.
Action: Search
Action Input: Eiffel Tower height in meters
Observation: No results found for: Eiffel Tower height in meters
Thought:Thought: I need to

'1063.52 feet'

## Implementation Notes and Best Practices

In [11]:
# Best practices for agent development
def agent_development_guidelines():
    """Print guidelines for developing effective agents"""
    guidelines = """
    AGENT DEVELOPMENT BEST PRACTICES:

    1. START SIMPLE: Begin with the simplest agent that can solve your problem
    2. TOOL SELECTION: Choose tools that match your specific use case
    3. ERROR HANDLING: Implement robust error handling for tool failures
    4. PROMPT ENGINEERING: Craft clear instructions and examples for the agent
    5. ITERATIVE TESTING: Test with diverse inputs to identify edge cases
    6. PERFORMANCE MONITORING: Track metrics like success rate and latency
    7. COST MANAGEMENT: Be mindful of API costs, especially with complex agents
    8. USER EXPERIENCE: Consider the trade-off between accuracy and response time

    AGENT SELECTION GUIDE:

    - Use ReAct agents for: Simple tasks, quick responses, straightforward problems
    - Use Plan-and-Execute agents for: Complex multi-step tasks, structured problems
    - Use Self-Ask agents for: Research tasks, fact verification, information gathering

    COMMON PITFALLS TO AVOID:

    1. Overcomplicating simple tasks with complex agents
    2. Insufficient error handling leading to agent failures
    3. Poor tool selection that doesn't match the task requirements
    4. Inadequate testing with edge cases and failure scenarios
    5. Ignoring cost implications of complex agent architectures
    """

    print(guidelines)

# Display the guidelines
agent_development_guidelines()

# Additional utility: Function to compare agent performance
def compare_agent_performance(tasks, agents, agent_names):
    """Compare the performance of different agents across multiple tasks"""
    results = {}

    for task in tasks:
        print(f"\n{'='*50}")
        print(f"TASK: {task}")
        print(f"{'='*50}")

        task_results = {}
        for name, agent in zip(agent_names, agents):
            print(f"\n{name} Agent:")
            try:
                result = agent.run(task)
                task_results[name] = {"result": result, "error": None}
                print(f"  Result: {result}")
            except Exception as e:
                task_results[name] = {"result": None, "error": str(e)}
                print(f"  Error: {e}")

        results[task] = task_results

    return results

# Test with multiple tasks
test_tasks = [
    "What is 15 * 24?",
    "Who is the current president of the United States?",
    "What is the square root of 144 plus 25?",
    "How far is the Moon from Earth in miles?"
]

performance_results = compare_agent_performance(test_tasks, agents, agent_names)


    AGENT DEVELOPMENT BEST PRACTICES:

    1. START SIMPLE: Begin with the simplest agent that can solve your problem
    2. TOOL SELECTION: Choose tools that match your specific use case
    3. ERROR HANDLING: Implement robust error handling for tool failures
    4. PROMPT ENGINEERING: Craft clear instructions and examples for the agent
    5. ITERATIVE TESTING: Test with diverse inputs to identify edge cases
    6. PERFORMANCE MONITORING: Track metrics like success rate and latency
    7. COST MANAGEMENT: Be mindful of API costs, especially with complex agents
    8. USER EXPERIENCE: Consider the trade-off between accuracy and response time

    AGENT SELECTION GUIDE:

    - Use ReAct agents for: Simple tasks, quick responses, straightforward problems
    - Use Plan-and-Execute agents for: Complex multi-step tasks, structured problems
    - Use Self-Ask agents for: Research tasks, fact verification, information gathering

    COMMON PITFALLS TO AVOID:

    1. Overcomplicating simple